## Training Simba model

Notebook to detail steps of training/fine-tuning/inference of the model used for Simba

In [1]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
import os

/home/freya/simplification/simp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/freya/simplification/simp_env/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
dataset = load_dataset('json', data_files='train_inst_dataset.jsonl')
dataset = dataset['train']
dataset.set_format("torch", device="cuda")

**Training datapoint example**

{"original": "Kannst du folgenden Zeitungsartikel vereinfachen: 'Ein Netzwerkfehler sorgte Dienstagvormittag für den Ausfall von 120 der 275 Lifte in den Wiener U-Bahn-Stationen. Die Störung sei in der Nacht auf Dienstag um 3.47 Uhr eingetreten, berichtete ORF Wien . Durch den Netzwerkfehler ist wenig später in den betroffenen Liften die Notruffunktion ausgefallen. Aus Sicherheitsgründen habe man die Aufzüge gestoppt, sagte eine Sprecherin der Wiener Linien. Nur jene Stationen, die mit Aufsichtspersonal besetzt sind, seien nicht betroffen, da dort keine Gefahr bestehe, dass Personen unbemerkt stecken blieben bzw. zu Schaden kämen, hieß es. Nach der Reparatur des Systems wurden die Aufzüge schrittweise wieder in Gang gesetzt. Um 10.30 Uhr waren laut Wiener Linien alle ausgefallenen Aufzüge wieder in Betrieb. Was zu dem Defekt führte, ist derzeit noch Gegenstand von Untersuchungen. Für die Fahrgäste war die Sperre zwar ärgerlich, besondere Vorfälle gab es aber nicht, bestätigte auch die Wiener Rettung auf Nachfrage des KURIER. Der nächste Ausfall ist geplant: Ab 30. April wird die U4 von Hütteldorf bis Hietzing gesperrt .'", "simplification": "In den Wiener U-Bahn-Stationen ist von 18. auf 19. April ein technischer-Fehler aufgetreten. Durch diesen Fehler sind in den betroffenen Aufzügen die Notrufsignale ausgefallen. Aus Sicherheitsgründen waren viele Aufzüge außer Betrieb. Das war in Stationen, wo es keine Aufsichts-Person gab. In den Stationen, wo es eine Aufsicht gibt, wurden keine Aufzüge gestoppt, da keine Gefahr für die Menschen war. Nach der Reparatur von dem technischen Problem wurden schrittweise die Aufzüge wieder in Betrieb genommen. Was der Auslöser für diesen Fehler war, wird noch untersucht. Für die Menschen war die Sperre zwar ärgerlich, aber zum Glück gab es keine Verletzten."}

In [3]:
base_model_name = "jphme/em_german_leo_mistral"
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    #quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map=device_map,
    #trust_remote_code=True,
    #use_auth_token=True
)
#base_model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.33s/it]


In [4]:
output_dir = "./mistral_fine_tuned_feb24"

training_args = TrainingArguments(output_dir=output_dir[2:], learning_rate=3e-5, warmup_steps=20, lr_scheduler_type="cosine", adam_beta1=0.9, adam_beta2=0.95)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

# More info: https://github.com/huggingface/transformers/pull/24906
#base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)#, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    logging_steps=10,
    max_steps=500
    #num_train_epochs=1
)

#(output_dir="mistral_fine_tune_1011", learning_rate=3e-5, warmup_steps=20, lr_scheduler_type="cosine", adam_beta1=0.9, adam_beta2=0.95)
#max_seq_length = 512

#Du bist ein hilfreicher Assistent. USER: <instruction> ASSISTANT:

def formatting_prompts_func(example):
    #print(example)
    output_texts = []
    for i in range(len(example['original'])):
        text = f"Du bist ein hilfreicher Assistent. USER: {example['original'][i]} ASSISTANT: {example['simplification'][i]}"
        output_texts.append(text)
    return output_texts

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    #max_seq_length=
    formatting_func=formatting_prompts_func,
    args=training_args,
)



/home/freya/simplification/simp_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [5]:
trainer.train()

Step,Training Loss
10,1.531500
20,1.454800
30,1.416600
40,1.334600
50,1.358100
60,1.386700
70,1.331900
80,1.292300
90,1.269600
100,1.313100


TrainOutput(global_step=500, training_loss=1.2665534152984619, metrics={'train_runtime': 1008.9171, 'train_samples_per_second': 1.982, 'train_steps_per_second': 0.496, 'total_flos': 7.254548169488794e+16, 'train_loss': 1.2665534152984619, 'epoch': 0.21})

In [ ]:
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

### Merging model

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
#import torch
from transformers import AutoModelForCausalLM

In [ ]:
adapter_model_name="/home/freya/simplification/code_for_training/mistral_fine_tuned_feb24/final_checkpoint/"
base_model_name = "jphme/em_german_leo_mistral"
model = AutoModelForCausalLM.from_pretrained(base_model_name)#, device_map="auto")
model = PeftModel.from_pretrained(model, adapter_model_name)#, #device_map="auto")

model = model.merge_and_unload()

In [ ]:
model.half()
model.save_pretrained("merged_model_mistral_fine_tuned_feb24")

In [ ]:
# Push to HF
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="merged_model_mistral_fine_tuned_feb24",
    repo_id="hiig-piai/...........",
    repo_type="model",
)


### Inference

In [ ]:
prompt_format = "Du bist ein hilfreicher Assistent. USER: Kannst du folgenden Zeitungsartikel vereinfachen: '" 
prompt_format2 = "' ASSISTANT: "

In [ ]:
text5 = """Schulen, Kitas, Bürgerämter: Am Mittwoch dürften viele Berlinerinnen und Berliner Einschränkungen bemerken - wegen eines Warnstreiks der Beschäftigten des öffentlichen Dienstes. Gewerkschaften fordern 10,5 Prozent mehr Lohn.

Notbetreuung in Schulen, geschlossene Kitas, eingeschränkte Besetzung der Bürgerämter: Am Mittwoch hat ein Warnstreik der Berliner Beschäftigten, die unter den Tarifvertrag der Länder fallen, begonnen. Es sei mit Unterrichtsausfall an zahlreichen Schulen zu rechnen, sagte ein Sprecher der beteiligten Gewerkschaft Erziehung und Wissenschaft (GEW) am Morgen. Mindestens 100 Kitas blieben geschlossen, sagte ein Verdi-Sprecher dem rbb.

Zum Warnstreik aufgerufen sind die Beschäftigten der Senatsverwaltungen und Bezirksämter, die Schulen und Hochschulen, die Polizeidienststellen, die Feuerwehr, die Kitas und die forstwirtschaftlichen Betriebe des Landes Berlin, wie die Gewerkschaften Verdi, Erziehung und Wissenschaft (GEW), der Polizei (GdP) und IG Bau gemeinsam mitteilten.

Begleitet wird der Streik in Berlin von einer Demo am Vormittag. Sie beginnt am Wittenbergplatz und zieht dann bis zum Platz des 18. März. Die Organisatoren rechnen eigenen Angaben zufolge mit Tausenden Teilnehmern. Auch in den anderen Stadtstaaten Bremen und Hamburg sind die Beschäftigten des öffentlichen Diensts zu Arbeitsniederlegungen aufgerufen.
"""

In [ ]:
# Cleaning output (this might be able to be done more cleverly, i.e. maybe there's some model params that do the same thing?)
# ensure output is not cut off mid-sentence
# ensure output doesn't have repetitive content

import re

def split_into_sentences(text):
  # from here: https://github.com/brjezierski/scrapers/blob/51da6fa87879217c5676df87a5f28873ee8e0826/preprocess.py#L88C1-L100C19 
 
  sentences = re.split(r"(?<!\w\.\w.)(?<![0-9]\.)(?<![0-9][0-9]\.)(?<![A-Z]\.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", text)
  sentences = [s for s in sentences if s]
  return sentences

from difflib import SequenceMatcher as SM

def remove_repetitive_output(raw_output):

    output_sents = split_into_sentences(raw_output.split("ASSISTANT: ")[1])

    outputs = ""
    sentences = list()

    for x in range(1,len(output_sents)):
        if SM(None,output_sents[x-1],output_sents[x]).ratio() < 0.6:
            if output_sents[x-1][-1] == ".":
                if output_sents[x-1].replace("\n", "") not in sentences:
                    outputs += output_sents[x-1].replace("\n", "") + " "
                    sentences.append(output_sents[x-1].replace("\n", ""))
    
    return outputs


In [ ]:
# Load model if not already loaded
output_dir="merged_model_mistral_fine_tuned_feb24/final_checkpoint/"
device_map = {"": 0}

model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)

base_model_name = "jphme/em_german_leo_mistral"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)#, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

### GPTQ quantization